In [ ]:
import re

class Bot:
    def __init__(self, id, start, destination):
        self.id = id
        self.position = start  # Starting position on the grid (row, col)
        self.destination = destination  # Destination position on the grid (row, col)
        self.commands = []  # Pathfinding commands (to be filled later)
        self.direction = 0  # Initial direction (0: North, 1: East, 2: South, 3: West)

    def forward(self):
        """Move forward based on current direction."""
        if self.direction == 0:  # North
            self.position = (self.position[0] - 1, self.position[1])
        elif self.direction == 1:  # East
            self.position = (self.position[0], self.position[1] + 1)
        elif self.direction == 2:  # South
            self.position = (self.position[0] + 1, self.position[1])
        elif self.direction == 3:  # West
            self.position = (self.position[0], self.position[1] - 1)

    def reverse(self):
        """Move backward based on current direction."""
        if self.direction == 0:  # North
            self.position = (self.position[0] + 1, self.position[1])
        elif self.direction == 1:  # East
            self.position = (self.position[0], self.position[1] - 1)
        elif self.direction == 2:  # South
            self.position = (self.position[0] - 1, self.position[1])
        elif self.direction == 3:  # West
            self.position = (self.position[0], self.position[1] + 1)

    def left(self):
        """Turn left (counterclockwise)."""
        self.direction = (self.direction - 1) % 4

    def right(self):
        """Turn right (clockwise)."""
        self.direction = (self.direction + 1) % 4

    def wait(self):
        """Wait without moving."""
        pass

    def pathfinding(self,grid, bots):
        """
        Placeholder for a pathfinding algorithm.
        It should generate a list of movement commands (forward, right, left, etc.).
        """

        if(self.position[0]+1 < grid.length())

        # Placeholder: for now, return an empty command list.


        return ""

def simulate(grid, bots):
    """
    Simulate bot movements over the grid for a given number of time steps.
    """
    total_time = 0
    while  all_destination_reach(bots)==False:
        for bot in bots:
            if bot.position == bot.destination:
               break
            else:
                bot.commands.append(bot.pathfinding(grid , bots))
        total_time += 1
    return total_time


def all_destination_reach(bots):
    for bot in bots:
        if bot.position != bot.destination:
            return False
    return True



grid = [
    ['A1', '.', '.', 'X', 'B1'],
    ['A3', 'X', '.', '.', '.'],
    ['.', '.', 'X', '.', '.'],
    ['.', '.', '.', 'B3', '.'],
    ['A2', 'X', '.', '.', 'B2']
]

bots = []
bot_destinations = {}

for i in range(len(grid)):
    for j in range(len(grid[0])):
        cell = grid[i][j]
        if cell[0] == 'A':
            bot_id = cell  # The bot ID is something like 'A1'
            bots.append(Bot(bot_id, (i, j), None))  # Destination will be assigned later
        elif cell[0] == 'B':  # Detect destinations (e.g., 'B1', 'B2')
            bot_id = 'A' + cell[1]  # The bot's ID that corresponds to this destination
            bot_destinations[bot_id] = (i, j)  # Save the destination for bot

# Step 2: Assign the destination to each bot based on the 'B' entries
for bot in bots:
    bot.destination = bot_destinations[bot.id]  # Set destination for the bot


tt = simulate(grid, bots)

# Optional: Print bot details to see how they are set up
for bot in bots:
    print(f"Bot {bot.id}: Start {bot.position}, Destination {bot.destination}, Commands: {bot.commands}, total time {tt}")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import random

# Actions the bot can take
ACTIONS = ['forward', 'left', 'right', 'wait']  # Define available actions
ACTION_MAP = {0: 'forward', 1: 'left', 2: 'right', 3: 'wait'}

class Bot:
    def __init__(self, id, start, destination, grid_size):
        self.id = id
        self.position = start  # Starting position (i, j)
        self.destination = destination  # Destination position
        self.direction = 0  # Facing direction (0: North, 1: East, 2: South, 3: West)
        self.grid_size = grid_size
        self.q_table = np.zeros((grid_size * grid_size * 4, len(ACTIONS)))  # Q-table initialization

    def get_state(self):
        """Returns a unique state ID based on the current position and direction."""
        return self.position[0] * self.grid_size + self.position[1] + self.direction * self.grid_size * self.grid_size

    def move(self, action):
        """Perform the action and update the bot's position and direction."""
        if action == 0:  # Forward
            if self.direction == 0 and self.position[0] > 0:  # North
                self.position = (self.position[0] - 1, self.position[1])
            elif self.direction == 1 and self.position[1] < self.grid_size - 1:  # East
                self.position = (self.position[0], self.position[1] + 1)
            elif self.direction == 2 and self.position[0] < self.grid_size - 1:  # South
                self.position = (self.position[0] + 1, self.position[1])
            elif self.direction == 3 and self.position[1] > 0:  # West
                self.position = (self.position[0], self.position[1] - 1)
        elif action == 1:  # Turn left
            self.direction = (self.direction - 1) % 4
        elif action == 2:  # Turn right
            self.direction = (self.direction + 1) % 4
        # 'wait' action does nothing (action == 3)

    def reward(self):
        """Calculate reward based on the current position."""
        if self.position == self.destination:
            return 100  # High reward for reaching destination
        elif grid[self.position[0]][self.position[1]] == 'X':
            return -10  # Penalty for hitting an obstacle
        else:
            # Negative reward for every move to encourage reaching the destination faster
            return -1

    def q_learning(self, episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
        """Q-learning algorithm."""
        for episode in range(episodes):
            self.position = (random.randint(0, self.grid_size - 1), random.randint(0, self.grid_size - 1))  # Random start
            self.direction = random.randint(0, 3)  # Random direction

            for step in range(100):  # Limit steps to avoid infinite loops
                state = self.get_state()

                # Epsilon-greedy action selection
                if random.uniform(0, 1) < epsilon:
                    action = random.randint(0, len(ACTIONS) - 1)  # Explore
                else:
                    action = np.argmax(self.q_table[state])  # Exploit

                # Move the bot based on the chosen action
                self.move(action)
                next_state = self.get_state()

                # Receive reward
                reward = self.reward()

                # Q-learning update rule
                best_next_action = np.argmax(self.q_table[next_state])
                self.q_table[state, action] += alpha * (reward + gamma * self.q_table[next_state, best_next_action] - self.q_table[state, action])

                # Check if the bot reached its destination
                if self.position == self.destination:
                    break

# Define grid
grid = [
    ['A1', '.', '.', 'X', 'B1'],
    ['.', 'X', '.', '.', '.'],
    ['.', '.', 'X', '.', '.'],
    ['.', '.', '.', '.', '.'],
    ['A2', 'X', '.', '.', 'B2']
]

grid_size = len(grid)  # Size of the grid

# Initialize Bot A1 with its start and destination
bot_a1 = Bot('A1', (0, 0), (0, 4), grid_size)

# Run Q-learning
bot_a1.q_learning(episodes=1000)

# Print learned Q-table
print("Q-table for Bot A1:")
print(bot_a1.q_table)


Q-table for Bot A1:
[[ 1.69258124e+01  1.79839117e+00  6.12934564e+01  1.08157942e+01]
 [ 2.65270550e+00 -5.49148501e-01  5.75140185e+01 -4.90099501e-01]
 [ 1.53131298e+01  1.76762070e+00  8.20862706e+01  1.44842666e+01]
 [-1.00000000e+00 -1.00000000e+00  7.93416128e+01  0.00000000e+00]
 [ 4.90099501e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 5.12422857e+01  1.92716547e+00 -1.39472394e+00 -1.39219834e+00]
 [ 2.01683886e+01 -1.90000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 6.29005421e+01  6.59649245e-01 -2.36067813e-01 -2.97010000e-01]
 [ 4.73473274e+00 -1.90000000e-01  4.69234249e+01  2.39552951e+00]
 [ 1.43757094e+02  5.67888424e+01  8.19904311e+01  9.81751072e+01]
 [ 3.32157723e+01 -1.78594349e+00 -1.41976351e+00 -1.82093062e+00]
 [-2.86173147e+00  4.12848353e+00 -2.13150505e+00 -2.14198658e+00]
 [ 2.16811012e+01 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 3.44913641e+00 -3.69910000e-01  2.73588932e+01 -2.97010000e-01]
 [ 1.26733166e+02  4.93886636e+01  5.40688

In [ ]:
import numpy as np
import random

class Bot:
    def __init__(self, id, start, destination):
        self.id = id
        self.position = start  # Starting position on the grid (row, col)
        self.destination = destination  # Destination position on the grid (row, col)
        self.q_table = {}  # Q-table for state-action values
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor
        self.epsilon = 1.0  # Exploration rate

    def get_state(self):
        return (self.position[0], self.position[1])

    def choose_action(self):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(["forward", "reverse", "left", "right", "wait"])
        else:
            return max(self.q_table.get(self.get_state(), {}), key=self.q_table.get(self.get_state(), {}).get, default="wait")

    def learn(self, state, action, reward, next_state):
        current_q = self.q_table.get(state, {}).get(action, 0)
        max_future_q = max(self.q_table.get(next_state, {}).values(), default=0)
        new_q = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)
        if state not in self.q_table:
            self.q_table[state] = {}
        self.q_table[state][action] = new_q

    def forward(self):
        self.position = (self.position[0] - 1, self.position[1])  # Move North

    def reverse(self):
        self.position = (self.position[0] + 1, self.position[1])  # Move South

    def left(self):
        self.position = (self.position[0], self.position[1] - 1)  # Move West

    def right(self):
        self.position = (self.position[0], self.position[1] + 1)  # Move East

    def wait(self):
        pass  # Stay in place

def get_reward(bot, grid):
    if bot.position == bot.destination:
        return 10  # Reward for reaching the destination
    elif 0 <= bot.position[0] < len(grid) and 0 <= bot.position[1] < len(grid[0]) and grid[bot.position[0]][bot.position[1]] == 'X':  # Check bounds before accessing grid
        return -10  # Penalty for hitting a wall
    else:
        return -1  # Small penalty for each step taken

def simulate(grid, bots, max_time):
    for episode in range(100):  # Number of training episodes
        for time in range(max_time):
            for bot in bots:
                current_state = bot.get_state()
                action = bot.choose_action()

                # Execute the action and update the bot's position
                if action == "forward":
                    bot.forward()
                elif action == "reverse":
                    bot.reverse()
                elif action == "left":
                    bot.left()
                elif action == "right":
                    bot.right()
                elif action == "wait":
                    bot.wait()

                next_state = bot.get_state()  # Update to the new state
                reward = get_reward(bot, grid)  # Get reward

                bot.learn(current_state, action, reward, next_state)

        # Decrease exploration rate over time
        if bot.epsilon > 0.1:
            bot.epsilon *= 0.995

# Define the grid and initialize bots
grid = [
    ['A1', '.', '.', 'X', 'B1'],
    ['A3', 'X', '.', '.', '.'],
    ['.', '.', 'X', '.', '.'],
    ['.', '.', '.', 'B3', '.'],
    ['A2', 'X', '.', '.', 'B2']
]

bots = []
bot_destinations = {}

for i in range(len(grid)):
    for j in range(len(grid[0])):
        cell = grid[i][j]
        if cell[0] == 'A':
            bot_id = cell
            bots.append(Bot(bot_id, (i, j), None))  # Create bot
        elif cell[0] == 'B':
            bot_id = 'A' + cell[1]  # Corresponding bot ID
            bot_destinations[bot_id] = (i, j)  # Save destination

# Assign destinations to bots
for bot in bots:
    bot.destination = bot_destinations[bot.id]  # Set destination for each bot

# Simulate the bots
max_time = 10
simulate(grid, bots, max_time)

# Print bot details
for bot in bots:
    print(f"Bot {bot.id}: Start {bot.position}, Destination {bot.destination}, Q-table: {bot.q_table}")


Bot A1: Start (22, -9), Destination (0, 4), Q-table: {(0, 0): {'left': -0.1}, (0, -1): {'wait': -0.1, 'reverse': -0.1}, (1, -1): {'left': -0.19981000000000002}, (1, -2): {'right': -0.10900000000000001, 'wait': -0.10981000000000002, 'reverse': -0.1}, (2, -2): {'left': -0.1}, (2, -3): {'reverse': -0.1}, (3, -3): {'reverse': -0.1}, (4, -3): {'left': -0.29927646100000005, 'wait': -0.31310566390000005}, (4, -4): {'right': -0.21608290000000002, 'forward': -0.1}, (3, -4): {'left': -0.1}, (3, -5): {'left': -0.1}, (3, -6): {'left': -0.1}, (3, -7): {'forward': -0.1}, (2, -7): {'wait': -0.1, 'left': -0.1}, (2, -8): {'left': -0.19981000000000002, 'forward': -0.1, 'reverse': -0.1}, (2, -9): {'left': -0.199, 'right': -0.10900000000000001, 'reverse': -0.20863900000000002, 'wait': -0.10981000000000002, 'forward': -0.29758510000000005}, (2, -10): {'forward': -0.288829, 'left': -0.36990051490000003, 'right': -0.10981000000000002}, (1, -10): {'reverse': -0.38380465900000005, 'right': -0.1, 'wait': -0.295

tenser flow